# Laboratório RAG com LangChain, Groq e Pinecone

Este notebook demonstra a implementação de um sistema Retrieval-Augmented Generation (RAG) utilizando a biblioteca LangChain, modelos de linguagem da Groq e o Pinecone como banco de dados vetorial. O objetivo é responder a perguntas com base em documentos PDF fornecidos, focando em atividades físicas e musculação.

## Configuração do Ambiente

### Importar bibliotecas

In [1]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import zipfile
import os
import shutil
import time

/home/kevin/anaconda3/envs/langchain/lib/python3.13/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


### Configuração das Chaves de API

In [2]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

True

## Carregando a LLM da Groq

In [3]:
llm = ChatGroq(model_name='llama-3.3-70b-versatile',temperature=0)

## Carregamento e Processamento dos Documentos PDF

In [4]:
zip_file_path = 'Arquivos.zip'
extracted_folder_path = 'docs'

### Extração dos PDFs do arquivo ZIP

In [5]:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

### Leitura dos Documentos PDF

In [6]:
documents = []
for filename in os.listdir(extracted_folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(extracted_folder_path, filename)
        loader = PyPDFLoader(file_path)
        documents.extend(loader.load())

In [7]:
# Antes de apagar, é uma boa prática verificar se o diretório existe
if os.path.exists(extracted_folder_path):
    print(f"Apagando o diretório e seu conteúdo: {extracted_folder_path}")
    try:
        # A função rmtree() remove o diretório e tudo que está dentro dele
        shutil.rmtree(extracted_folder_path)
        print("Diretório apagado com sucesso.")
    except OSError as e:
        print(f"Erro: {e.strerror}")
else:
    print(f"O diretório '{extracted_folder_path}' não existe.")

Apagando o diretório e seu conteúdo: docs
Diretório apagado com sucesso.


## Geração de Embeddings e Armazenamento no Pinecone

### Divisão dos Documentos em Chunks

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)


In [9]:
print(f'Total de chunks: {len(docs)}')

Total de chunks: 39


In [10]:
docs[1]

Document(metadata={'producer': 'Skia/PDF m128', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/128.0.0.0 Safari/537.36', 'creationdate': '2025-09-03T18:30:27+00:00', 'title': 'principios_treinamento_musculacao.md', 'moddate': '2025-09-03T18:30:27+00:00', 'source': 'docs/principios_treinamento_musculacao.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='corpo se adapta e o progresso estagna.\nPrincípio da Especiﬁcidade\nO corpo se adapta especiﬁcamente ao tipo de estresse a que é submetido. Isso signiﬁca\nque, se o objetivo é aumentar a força em um determinado movimento, o treinamento\ndeve focar nesse movimento ou em variações próximas. Da mesma forma, se o objetivo\né hipertroﬁa (ganho de massa muscular), o treino deve ser estruturado para maximizar\no estresse metabólico e a tensão mecânica nos músculos [\x00].\nPrincípio da Individualidade\nCada indivíduo responde de forma diferente ao treinamento devido a fatores

### Inicializar modelo de embeddings (HuggingFace para uso local/gratuito)

In [11]:

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/tmp/ipykernel_36941/2759443027.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Inicializar Pinecone

In [12]:
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [13]:
index_name = "rag-musculacao" 
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()] # change if desired

### Deletar e recriar o índice para garantir que a dimensão esteja correta

In [14]:

if index_name in existing_indexes:
    print(f"Deletando o índice existente '{index_name}'...")
    pc.delete_index(index_name)
    time.sleep(1) # Aguardar a exclusão

In [15]:
pc.create_index(
    name=index_name,
    metric="cosine",
    dimension=384, # Isso irá retornar 384
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)
while not pc.describe_index(index_name).status["ready"]:
    print("Aguardando o índice ficar pronto..."+pc.describe_index(index_name).status)
    time.sleep(1)

index = pc.Index(index_name)

### Criar ou conectar ao VectorStore do Pinecone

In [16]:
vectorstore = PineconeVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    index_name=index_name,
)

## Construção da Cadeia RAG

#### Definir o prompt para a LLM

In [17]:

prompt = ChatPromptTemplate.from_template("""Responda à pergunta com base apenas no contexto fornecido.
Se você não souber a resposta, diga que não sabe, não tente inventar uma resposta.

Contexto: {context}

Pergunta: {input}""")

### Criar a cadeia de documentos

In [18]:

document_chain = create_stuff_documents_chain(llm, prompt)



### Criar a cadeia de recuperação

In [19]:

retrieval_chain = create_retrieval_chain(vectorstore.as_retriever(), document_chain)

## Testando o Sistema RAG

In [20]:
# Função para fazer perguntas
def ask_question(question):
    response = retrieval_chain.invoke({"input": question})
    print(f"Pergunta: {question}")
    print(f"Resposta: {response['answer']}")
    

In [21]:
#Exemplos de perguntas (use os prompts e perguntas de teste )

pergunta1 = "Quais são os principais benefícios da atividade física regular para a saúde cardiovascular?"

pergunta2 = "Explique o princípio da sobrecarga progressiva na musculação."

pergunta3 = "Qual a importância dos carboidratos na nutrição para atividade física?"

pergunta4 = "Cite três exercícios essenciais para membros superiores e seus principais músculos trabalhados."

pergunta5 = "O que é periodização do treinamento e quais são seus principais ciclos?"

pergunta6 = "Quais são as principais estratégias para prevenir lesões na musculação?"

In [22]:
ask_question(pergunta3)

Pergunta: Qual a importância dos carboidratos na nutrição para atividade física?
Resposta: Os carboidratos são a principal fonte de energia para o exercício, especialmente para atividades de alta intensidade e longa duração. Eles são armazenados nos músculos e no fígado na forma de glicogênio. A ingestão adequada de carboidratos antes, durante (para exercícios prolongados) e após o treino é fundamental para manter os níveis de energia e otimizar a recuperação.


## Limpeza (Opcional)

In [ ]:
# Para deletar o índice do Pinecone (use com cautela!)
# pc.delete_index(index_name)